In [77]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

In [78]:
import pandas as pd

In [79]:
def save_answers(fn,accuracy):
     with open(fn, "w") as fout:
        fout.write(str(accuracy))

def save_answers_list(fn, l):
    with open(fn, "w") as fout:
        fout.write(" ".join([str(el) for el in l]))

In [80]:
#вывод на экран содержимого файла
with open('SMSSpamCollection.txt', 'r') as file_obj:
    print file_obj.readline()
    print file_obj.readline()
    print file_obj.readline()
    print file_obj.readline()
    file_obj.close()

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

ham	Ok lar... Joking wif u oni...

spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's

ham	U dun say so early hor... U c already then say...



In [81]:
d = pd.read_csv('SMSSpamCollection.txt', sep = '\t', header = None, names = ['flag','txt'])

In [82]:
d.head()

,flag,txt
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [83]:
d['class'] = 0
d.loc[d.flag == 'spam','class'] = 1

In [84]:
d.head()

,flag,txt,class
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [85]:
txt = d.txt
cls = d['class']

Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [86]:
v = CountVectorizer()
X = v.fit_transform(txt)
print(X.shape)

(5572, 8713)


Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [87]:
est = LogisticRegression(random_state = 2)
cv_rez = cross_val_score(est, X, cls, cv = 10, scoring = 'f1')
print cv_rez

[0.95890411 0.89855072 0.91549296 0.95833333 0.93706294 0.91304348
 0.94444444 0.92753623 0.92198582 0.95104895]


In [88]:
print cv_rez.mean()
print "%.1f"%cv_rez.mean()

0.9326402983610631
0.9


In [89]:
save_answers('1.txt', "%.1f"%cv_rez.mean())

 А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [90]:
est = LogisticRegression(random_state = 2)
est.fit(X, cls)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [91]:
txt_new = ("FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"
           ,"FreeMsg: Txt: claim your reward of 3 hours talk time"
           ,"Have you visited the last lecture on physics?"
           ,"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"
           ,"Only 99$")
X_new = v.transform(txt_new)
print X_new.shape
print X.shape

(5, 8713)
(5572, 8713)


In [92]:
prd = est.predict(X_new)
print prd

save_answers_list('2.txt', prd)

[1 1 0 0 0]


Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [93]:
def support_return_vect(v):
    X = v.fit_transform(txt)
    est = LogisticRegression(random_state = 2)
    cv_rez = cross_val_score(est, X, cls, cv = 10, scoring = 'f1')
    return cv_rez

In [94]:
v = (CountVectorizer(ngram_range=(2,2))
     ,CountVectorizer(ngram_range=(3,3))
     ,CountVectorizer(ngram_range=(1,3)))
rez_list = []
for i in range(3):
    rez = support_return_vect(v[i])
    print rez.mean(), "%.2f"%rez.mean()
    rez_list.append("%.2f"%rez.mean())

print rez_list
save_answers_list('3.txt', rez_list)

0.8224220664187133 0.82
0.7250161555467377 0.73
0.9251382558648837 0.93
['0.82', '0.73', '0.93']


Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

In [95]:
def support_return_vect_MultinomialNB(v):
    X = v.fit_transform(txt)
    est = MultinomialNB()
    cv_rez = cross_val_score(est, X, cls, cv = 10, scoring = 'f1')
    return cv_rez

In [96]:
v = (CountVectorizer(ngram_range=(2,2))
     ,CountVectorizer(ngram_range=(3,3))
     ,CountVectorizer(ngram_range=(1,3)))
rez_list = []
for i in range(3):
    rez = support_return_vect_MultinomialNB(v[i])
    print rez.mean(), "%.2f"%rez.mean()
    rez_list.append("%.2f"%rez.mean())

print rez_list
save_answers_list('4.txt', rez_list)

0.6455015177985443 0.65
0.37871948524573595 0.38
0.8884859656061002 0.89
['0.65', '0.38', '0.89']


 Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [97]:
v =  TfidfVectorizer()
rez1 = support_return_vect(v)
print rez1.mean()

v = CountVectorizer()
rez2 = support_return_vect(v)
print rez2.mean()

print rez2.mean() - rez1.mean()

0.8528599554172456
0.9326402983610631
0.07978034294381753


In [98]:
save_answers('5.txt', 0)